# SQLAlchemy ORM (Object Relational Model)

>[SQLAlchemy ORM (Object Relational Model)](#scrollTo=u_S8klCJ9Kr1)

>>[SQLAlchemy](#scrollTo=RhlL6qqI-EpH)

>>[Arquitectura](#scrollTo=cPwUxnc29Kr2)

>>[Declaración de modelos en el ORM](#scrollTo=YFS0CmuD9Kr3)

>>[Creación de una máquina](#scrollTo=EOhCmMsH9Kr4)

>>>[Emisión CREATE TABLE DDL](#scrollTo=P4SXGUEU9Kr4)

>>[Creación de objetos y persistencia](#scrollTo=KhPPISNj9Kr4)

>>[SELECT](#scrollTo=SI7vq_9M9Kr5)

>>[SELECT ... JOIN](#scrollTo=wnxwFB7A9Kr5)

>>[Actualización o adición de datos](#scrollTo=mYctr47-9Kr6)

>>[Borrado de registros](#scrollTo=Y0rbeNq19Kr6)



## SQLAlchemy

**SQLAlchemy** es una biblioteca de Python que proporciona una forma conveniente para interactuar con **bases de datos relacionales**. Permite al usuario trabajar con diferentes tipos de bases de datos, incluyendo PostgreSQL, MySQL, SQLite, Oracle, entre otras.

Una de sus ventajas es que permite trabajar con una **API unificada**, independientemente del tipo de base de datos que esté usando. Esto significa que el usuario puede escribir código que sea compatible con múltiples tipos de bases de datos sin tener que preocuparse por los detalles de implementación específicos de cada una.

Otra ventaja importante es que **permite trabajar con objetos de Python en lugar de tener que escribir consultas SQL** directamente. Esto hace que el código sea más fácil de entender y mantener.

Además, proporciona herramientas para trabajar con transacciones, esquemas de datos, modelado de objetos y más.

En resumen, es una biblioteca muy poderosa para trabajar con **bases de datos relacionales** en Python.

## Arquitectura

* El ORM se basa en una vista de la base de datos centrada en el dominio que es orientado a estados.

* El Core y el SQL se basa en una vista de la base de datos centrada en el esquema que es orientada a comandos.

![SQLAlchemy ORM Architecture](https://jdvelasq.github.io/courses/_images/sqla_arch_small.png)

## Declaración de modelos en el ORM

In [1]:
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.orm import declarative_base, relationship

# Se usará un enfoque declarativo para definir
# los modelos de base de datos
Base = declarative_base()

class User(Base):

    """
    Declaración de la entidad usuario
    """
    
    # Nombre de la tabla
    __tablename__ = "user_account"

    # Atributos
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)

    # Se establece un atributo "relación", que vinculará
    # el atributo a otra tabla.

    # Un User podrá tener muchas Address

    addresses = relationship(
        "Address", back_populates="user", cascade="all, delete-orphan"
    )

    # back_populates="user":
    # Indica que también se creará un atributo "user"
    # en la clase de modelo "Address", que se refiere 
    # al usuario asociado con cada dirección.

    # cascade="all, delete-orphan":
    # Establece un comportamiento de cascada en la relación, 
    # lo que significa que si se elimina un usuario, 
    # todas sus direcciones relacionadas también se eliminarán.

    def __repr__(self):
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"


class Address(Base):

    __tablename__ = "address"
    
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    
    user_id = Column(Integer, ForeignKey("user_account.id"), nullable=False)
    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

## Creación de una máquina

A continuación, la función `create_engine` se utiliza para crear un motor (engine) para una base de datos SQLite en memoria.

- `"sqlite+pysqlite:///:memory:"`: especifica el tipo de base de datos (SQLite) y la ubicación (en memoria) de la base de datos. 
  
  La sintaxis `sqlite+pysqlite` indica que se está usando la biblioteca pysqlite como controlador de base de datos para SQLite.
  
  El `:memory:` al final indica que la base de datos se creará en la memoria.

- `echo=False`: desactiva la impresión de mensajes en la consola del motor sobre las operaciones realizadas en la base de datos.

- `future=True`: indica que se utilizará el sistema de ejecución de sentencias SQL basado en Future (o "futuro"), el cual es una característica experimental en SQLAlchemy que permite escribir código más limpio y eficiente.

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(
    "sqlite+pysqlite:///:memory:",
    echo=False,
    future=True,
)

### Emisión CREATE TABLE DDL

In [ ]:
Base.metadata.create_all(engine)

## Creación de objetos y persistencia

In [ ]:
from sqlalchemy.orm import Session

with Session(engine) as session:

    spongebob = User(
        name="spongebob",
        fullname="Spongebob Squarepants",
        addresses=[Address(email_address="spongebob@sqlalchemy.org")],
    )

    sandy = User(
        name="sandy",
        fullname="Sandy Cheeks",
        addresses=[
            Address(email_address="sandy@sqlalchemy.org"),
            Address(email_address="sandy@squirrelpower.org"),
        ],
    )

    patrick = User(
        name="patrick",
        fullname="Patrick Star",
    )

    session.add_all(
        [
            spongebob,
            sandy,
            patrick,
        ]
    )

    session.commit()

## SELECT

In [ ]:
from sqlalchemy import select

session = Session(engine)

stmt = select(User).where(User.name.in_(["spongebob", "sandy"]))

for user in session.scalars(stmt):
    print(user)

User(id=1, name='spongebob', fullname='Spongebob Squarepants')
User(id=2, name='sandy', fullname='Sandy Cheeks')


## SELECT ... JOIN

La consulta a continuación busca la dirección de correo electrónico de Sandy en la tabla de direcciones de la base de datos a través de una relación con la tabla de usuarios.

Esto se logra mediante la construcción de una sentencia SQL SELECT en varias partes encadenadas juntas.

- `select(Address)`: Crea un objeto de selección SQLAlchemy que especifica que la consulta debe seleccionar de la tabla "Address" en la base de datos.

- `.join(Address.user)`: Especifica que la consulta debe unir la tabla "Address" con la tabla "User" a través de la relación definida en el modelo (es decir, la relación "user" en la clase "Address").

- Los `where` especifican las condiciones de búsqueda.

- `session.scalars(stmt).one()`: Esta función ejecuta la consulta construida anteriormente en la sesión de SQLAlchemy y devuelve un solo resultado. El resultado se devuelve como un objeto de dirección de correo electrónico.

In [ ]:
stmt = (
    select(Address)
    .join(Address.user)
    .where(User.name == "sandy")
    .where(Address.email_address == "sandy@sqlalchemy.org")
)

sandy_address = session.scalars(stmt).one()
sandy_address

Address(id=2, email_address='sandy@sqlalchemy.org')

## Actualización o adición de datos

In [ ]:
#
# SELECT ... WHERE
#
stmt = select(User).where(User.name == "patrick")

#
# Extrae el registro
#
patrick = session.scalars(stmt).one()

#
# Agrega nueva información
#
patrick.addresses.append(Address(email_address="patrickstar@sqlalchemy.org"))

sandy_address.email_address = "sandy_cheeks@sqlalchemy.org"

session.commit()

## Borrado de registros

In [ ]:
#
# Borra un campo en particular
#
sandy = session.get(User, 2)
sandy.addresses.remove(sandy_address)

In [ ]:
#
# Emite el cambio sin el commit()
#
session.flush()

In [ ]:
#
# Borra un registro completo
#
session.delete(patrick)

In [ ]:
#
# Emite los cambios
#
session.commit()